<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_03_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/chinook.db"

--2021-09-10 18:50:40--  https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/chinook.db
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/chinook.db [following]
--2021-09-10 18:50:41--  https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/chinook.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 884736 (864K) [application/octet-stream]
Saving to: ‘chinook.db’

chinook.db          100%[===================>] 864.00K  --.-KB/s    in 0.06s   

2021-09-10 18:50:41 (14.7 MB/s) - ‘chinook.db’ saved [884736/884736]



In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

tabelas = pd.read_sql_query(sql='SELECT name FROM sqlite_master WHERE type="table"',con=conn)
tabelas

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


<img src="https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg">

In [5]:
consulta = pd.read_sql_query(sql='SELECT * FROM customers',con=conn)
print('Registros =',len(consulta))
consulta.head()

Registros = 59


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [14]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          STRFTIME('%d/%m/%Y',y.InvoiceDate) as data_compra
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
      """,
      con = conn)
consulta.head()

,nome,pais,data_compra
0,Luís Gonçalves,Brazil,11/03/2010
1,Luís Gonçalves,Brazil,13/06/2010
2,Luís Gonçalves,Brazil,15/09/2010
3,Luís Gonçalves,Brazil,06/05/2011
4,Luís Gonçalves,Brazil,27/10/2012


In [17]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          STRFTIME('%d/%m/%Y',y.InvoiceDate) as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
      """,
      con = conn)
print('Registros =',len(consulta))
consulta.head()

Registros = 2240


,nome,pais,data_compra,preco_unitario,quantidade
0,Luís Gonçalves,Brazil,11/03/2010,1.99,1
1,Luís Gonçalves,Brazil,11/03/2010,1.99,1
2,Luís Gonçalves,Brazil,13/06/2010,0.99,1
3,Luís Gonçalves,Brazil,13/06/2010,0.99,1
4,Luís Gonçalves,Brazil,13/06/2010,0.99,1


In [21]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          y.InvoiceDate as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade,
          t.Name as faixa_nome,
          a.Title as album_nome,
          b.Name as artista,
          m.Name as formato_media
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      """,
      con = conn)
print('Registros =',len(consulta))
consulta.head(10)

Registros = 2240


,nome,pais,data_compra,preco_unitario,quantidade,faixa_nome,album_nome,artista,formato_media
0,Luís Gonçalves,Brazil,2010-03-11 00:00:00,1.99,1,Experiment In Terra,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),Protected MPEG-4 video file
1,Luís Gonçalves,Brazil,2010-03-11 00:00:00,1.99,1,Take the Celestra,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),Protected MPEG-4 video file
2,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Shout It Out Loud,Greatest Kiss,Kiss,MPEG audio file
3,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Calling Dr. Love,Greatest Kiss,Kiss,MPEG audio file
4,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Strutter,Greatest Kiss,Kiss,MPEG audio file
5,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Cold Gin,Greatest Kiss,Kiss,MPEG audio file
6,Luís Gonçalves,Brazil,2010-09-15 00:00:00,0.99,1,Think About You,Appetite for Destruction,Guns N' Roses,Protected AAC audio file
7,Luís Gonçalves,Brazil,2010-09-15 00:00:00,0.99,1,Rocket Queen,Appetite for Destruction,Guns N' Roses,Protected AAC audio file
8,Luís Gonçalves,Brazil,2010-09-15 00:00:00,0.99,1,Don't Cry (Original),Use Your Illusion I,Guns N' Roses,Protected AAC audio file
9,Luís Gonçalves,Brazil,2010-09-15 00:00:00,0.99,1,Back off Bitch,Use Your Illusion I,Guns N' Roses,Protected AAC audio file


In [22]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          y.InvoiceDate as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade,
          t.Name as faixa_nome,
          a.Title as album_nome,
          b.Name as artista,
          m.Name as formato_media
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      WHERE x.Country = 'Brazil' 
      """,
      con = conn)
print('Registros =',len(consulta))
consulta

Registros = 190


,nome,pais,data_compra,preco_unitario,quantidade,faixa_nome,album_nome,artista,formato_media
0,Luís Gonçalves,Brazil,2010-03-11 00:00:00,1.99,1,Experiment In Terra,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),Protected MPEG-4 video file
1,Luís Gonçalves,Brazil,2010-03-11 00:00:00,1.99,1,Take the Celestra,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),Protected MPEG-4 video file
2,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Shout It Out Loud,Greatest Kiss,Kiss,MPEG audio file
3,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Calling Dr. Love,Greatest Kiss,Kiss,MPEG audio file
4,Luís Gonçalves,Brazil,2010-06-13 00:00:00,0.99,1,Strutter,Greatest Kiss,Kiss,MPEG audio file
...,...,...,...,...,...,...,...,...,...
185,Fernanda Ramos,Brazil,2012-11-01 00:00:00,0.99,1,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Protected AAC audio file
186,Fernanda Ramos,Brazil,2012-11-01 00:00:00,0.99,1,Fast As a Shark,Restless and Wild,Accept,Protected AAC audio file
187,Fernanda Ramos,Brazil,2012-11-01 00:00:00,0.99,1,Snowballed,For Those About To Rock We Salute You,AC/DC,MPEG audio file
188,Fernanda Ramos,Brazil,2012-11-01 00:00:00,0.99,1,Go Down,Let There Be Rock,AC/DC,MPEG audio file


In [24]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          y.InvoiceDate as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade,
          t.Name as faixa_nome,
          a.Title as album_nome,
          b.Name as artista,
          m.Name as formato_media
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      ORDER BY b.Name DESC
      """,
      con = conn)
consulta.head(10)

,nome,pais,data_compra,preco_unitario,quantidade,faixa_nome,album_nome,artista,formato_media
0,František Wichterlová,Czech Republic,2012-09-05 00:00:00,0.99,1,Coração Em Desalinho,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
1,František Wichterlová,Czech Republic,2012-09-05 00:00:00,0.99,1,Camarão que Dorme e Onda Leva,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
2,Victor Stevens,USA,2011-05-29 00:00:00,0.99,1,Faixa Amarela,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
3,Victor Stevens,USA,2011-05-29 00:00:00,0.99,1,Sem Essa de Malandro Agulha,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
4,Victor Stevens,USA,2011-05-29 00:00:00,0.99,1,Verdade,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
5,Terhi Hämäläinen,Finland,2013-12-14 00:00:00,0.99,1,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
6,Terhi Hämäläinen,Finland,2013-12-14 00:00:00,0.99,1,Samba pras moças,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
7,Ladislav Kovács,Hungary,2010-02-18 00:00:00,0.99,1,Seu Balancê,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
8,Ladislav Kovács,Hungary,2010-02-18 00:00:00,0.99,1,Sapopemba e Maxambomba,Ao Vivo [IMPORT],Zeca Pagodinho,MPEG audio file
9,Isabelle Mercier,France,2012-10-06 00:00:00,0.99,1,"Suite for Solo Cello No. 1 in G Major, BWV 100...",Bach: The Cello Suites,Yo-Yo Ma,Protected AAC audio file


In [32]:
consulta = pd.read_sql_query(
    sql= """
      SELECT
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          SUM(z.UnitPrice * z.Quantity) as total_pais,
          COUNT(*) as num_faixas
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
      WHERE x.Country = 'Brazil'
      GROUP BY x.Firstname || ' ' || x.LastName, x.Country
      ORDER BY SUM(z.UnitPrice * z.Quantity) DESC, (x.Firstname || ' ' || x.LastName) DESC
      """,
      con = conn)
consulta

,nome,pais,total_pais,num_faixas
0,Luís Gonçalves,Brazil,39.62,38
1,Roberto Almeida,Brazil,37.62,38
2,Fernanda Ramos,Brazil,37.62,38
3,Eduardo Martins,Brazil,37.62,38
4,Alexandre Rocha,Brazil,37.62,38


In [35]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Country as pais,
          COUNT(*) as num_zeca
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      WHERE B.Name = 'Zeca Pagodinho'
      GROUP BY x.Country
      ORDER BY COUNT(*) DESC
      """,
      con = conn)
consulta.head(10)

,pais,num_zeca
0,USA,3
1,Czech Republic,2
2,Finland,2
3,Hungary,2


In [40]:
consulta = pd.read_sql_query(
    sql= """
      SELECT
          a.Title as album_nome,
          b.Name as artista,
          COUNT(*) as num_faixas
      FROM tracks as t INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      GROUP BY a.Title, b.Name
      ORDER BY COUNT(*) DESC
      LIMIT 10
      """,
      con = conn)
consulta

,album_nome,artista,num_faixas
0,Greatest Hits,Lenny Kravitz,57
1,Minha Historia,Chico Buarque,34
2,Unplugged,Eric Clapton,30
3,"Lost, Season 3",Lost,26
4,"Lost, Season 1",Lost,25
5,"The Office, Season 3",The Office,25
6,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),24
7,"Lost, Season 2",Lost,24
8,My Way: The Best Of Frank Sinatra [Disc 1],Frank Sinatra,24
9,Afrociberdelia,Chico Science & Nação Zumbi,23
